# Importing the necessary libraries

In [5]:
pip install numpy

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.22.3.zip (11.5 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing
import seaborn as sns 
import matplotlib.pyplot as plt
import missingno

ModuleNotFoundError: No module named 'numpy'

# Import the data to process it

We import the training data and the test data, we will only be working with the training data. Ultimately, we will make the predictions about the test data

In [ ]:
# concatenating the training and the testing data so we have more values to work with and therefor we can better approximate the missing values
# the combined data differentiates the training set and the test set by encoding them with 1 and 0 respectively
training = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
training['train_test'] = 1
test['train_test'] = 0
test['Survived'] = np.NaN
all_data = pd.concat([training,test])

%matplotlib inline
all_data.columns

# 1. Data analysis 
### 1.1 Visulisation and properties

In [ ]:
# determine the null counts (missing data)
training. info()

In [ ]:
# Missing data in training set

missingno.matrix(training)

In [ ]:
# determine the central dendencies of the data
training.describe()

In [ ]:
# Seperate the numerical and the categorical data  so different visualisation techniques can be used

df_numeric = training[['Age','SibSp','Parch','Fare']]
df_catergorical = training[['Survived','Pclass','Sex','Ticket','Cabin','Embarked']]

In [ ]:
# for the numerical values look at their distributions
for i in df_numeric.columns:
    plt.hist(df_numeric[i])
    plt.title(i)
    plt.show()

In [ ]:
# determine the correlations of the numerical data using a heat map

sns.heatmap(df_numeric[['Fare','Parch','SibSp','Age']].corr(), annot = True, fmt = '.2f', cmap = 'coolwarm')

In [ ]:
### DONNO WHAT THIS IS SHOWING US TRY UNDERSTAND
# To get a feel for the survival rates we will compare the survival rate for Age, Fare, Parch and SibSp

pd.pivot_table(training, index = 'Survived', values = ['Age','SibSp','Parch','Fare'])

In [ ]:
# make bar charts to understand the spread of the classes

for i in df_catergorical.columns:
    sns.barplot(df_catergorical[i].value_counts().index,df_catergorical[i].value_counts()).set_title(i);
    plt.show()
    

In [ ]:
# To get a feel for the survival rates we will look at the survival rate for the categories in Pclass, Sex and Embarked

print(pd.pivot_table(training, index = 'Survived', columns = 'Pclass', values = 'Ticket' ,aggfunc ='count'))
print()
print(pd.pivot_table(training, index = 'Survived', columns = 'Sex', values = 'Ticket' ,aggfunc ='count'))
print()
print(pd.pivot_table(training, index = 'Survived', columns = 'Embarked', values = 'Ticket' ,aggfunc ='count'))

In [ ]:
# Value counts of the sex column

training['Sex'].value_counts(dropna = False)
print('-'*40)
# Comment: There are more male passengers than female passengers on titanic
# Mean of survival by sex
training[['Sex', 'Survived']].groupby('Sex', as_index = False).mean().sort_values(by = 'Survived', ascending = False)

In [ ]:
sns.barplot(x = 'Sex', y ='Survived', data = training)
plt.ylabel('Survival Probability')
plt.title('Survival Probability by Gender')

# Comment: Female passengers are more likely to survive

In [ ]:
# Value counts of the Pclass column 

training['Pclass'].value_counts(dropna = False)
print('-'*40)

# Mean of survival by passenger class

training[['Pclass', 'Survived']].groupby(['Pclass'], as_index = False).mean().sort_values(by = 'Survived', ascending = False)


In [ ]:
sns.barplot(x = 'Pclass', y ='Survived', data = training)
plt.ylabel('Survival Probability')
plt.title('Survival Probability by Passenger Class')

# Comment: Survival probability decrease with passenger class, first class passengers are prioritised during evacuation

In [ ]:
# Survival by gender and passenger class

g = sns.factorplot(x = 'Pclass', y = 'Survived', hue = 'Sex', data = training, kind = 'bar')
g.despine(left = True)
plt.ylabel('Survival Probability')
plt.title('Survival Probability by Sex and Passenger Class')

In [ ]:
# Value counts of the Embarked column 

training['Embarked'].value_counts(dropna = False)
# Comment: Two missing values in the Embarked column
print('-'*40)
# Mean of survival by point of embarkation

training[['Embarked', 'Survived']].groupby(['Embarked'], as_index = False).mean().sort_values(by = 'Survived', ascending = False)


### 1.2 Feature engineering
The data for the cabin and tickes has many classes and is very messy therefore we will try clean it up by leveraging these classes to create new variables that aren't in the training set, Furthermore we will look at if the persons title is related to that persons survival

#### Cabin data

Determine if the cabin's letter had any impact on survival as well as determine if purchasing more thn one cabin had an impact on survivalDetermine if the cabin's letter had any impact on survival as well as determine if purchasing more thn one cabin had an impact on survival

In [ ]:
# if the value is 0 it means that the cabin number is missing
# if the number is 1 it means that the passanger had one cain and if the number is 2 it means the passenger has 2 cabins ect
df_catergorical.Cabin
training['number_of_cabins'] = training.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))
print(training['number_of_cabins'].value_counts())


pd.pivot_table(training, index = 'Survived', columns = 'number_of_cabins', values = 'Ticket' ,aggfunc ='count')

In [ ]:
# Split up the cabins into cabin letters and cabin letter to see if there is link to survival
# n stands for null value meaining that there is a missing value or the cabin  does not have a letter
training['cabin_letter'] = training.Cabin.apply(lambda x: str(x)[0])
print(training.cabin_letter.value_counts())
pd.pivot_table(training,index='Survived',columns='cabin_letter', values = 'Name', aggfunc='count')


#### Ticket data
looking at the numerical values on the ticket and the letters on the tickets and comparing them to survival

In [ ]:
# Look at numeric vs non numeric tickecks
training['numeric_ticket'] = training.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
training['ticket_letters'] = training.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1]).replace('.','').replace('/','').lower() if len(x.split(' ')[:-1]) >0 else 0)

# 0 represents numeric numbers
# 1 represents non numeric numbers

training['numeric_ticket'].value_counts()

In [ ]:
# comparing the numeric and non numeric tickets to the survival rate
# 0 represents numeric numbers
# 1 represents non numeric numbers
pd.pivot_table(training,index='Survived',columns='numeric_ticket', values = 'Ticket', aggfunc='count')

In [ ]:
# looking at the different types of tickets basked on the ticket letters and compare them to the survival rateticket_letters
# 0 means that the ticket does not have a letter 
pd.pivot_table(training,index='Survived',columns='ticket_letters', values = 'Ticket', aggfunc='count')

#### Title data

In [ ]:
training.Name.head()
# splits up the data into the persons title 
training['title'] = training.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())
training['title'].value_counts()

In [ ]:
# grouping ages 
# bins = [0,16,21,26,31,36,41,46,51,56,61,66,71,76,81,86]
# labels = ['0-15','16-20','21-25','26-30','31-35','36-40','41-45','46-50','51-55','56-60','61-65','71-75','76-80','81-85','86-x']
# # labels = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
# train_copy['Age'] = pd.cut(train_copy['Age'], bins=bins, labels=labels, right=False)
# train_copy.head()

#### Family Size

In [ ]:
# feature engineering 
training['FamilySize']=training['Parch']+training['SibSp']+1
training=training.drop(['SibSp'],axis=1)
training=training.drop(['Parch'],axis=1)
training.head()

In [ ]:
# Value counts of the Family column 

training['FamilySize'].value_counts(dropna = False)
print('-'*40)
training[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index = False).mean().sort_values(by = 'Survived', ascending = False)

### 1.3Data Preprocessing


## Remove outliers

In [ ]:
# plagiarised

def detect_outliers(df, n, features):
    """"
    This function will loop through a list of features and detect outliers in each one of those features. In each
    loop, a data point is deemed an outlier if it is less than the first quartile minus the outlier step or exceeds
    third quartile plus the outlier step. The outlier step is defined as 1.5 times the interquartile range. Once the 
    outliers have been determined for one feature, their indices will be stored in a list before proceeding to the next
    feature and the process repeats until the very last feature is completed. Finally, using the list with outlier 
    indices, we will count the frequencies of the index numbers and return them if their frequency exceeds n times.    
    """
    outlier_indices = [] 
    for col in features: 
        Q1 = np.percentile(df[col], 25)
        Q3 = np.percentile(df[col], 75)
        IQR = Q3 - Q1
        outlier_step = 1.5 * IQR 
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step)].index
        outlier_indices.extend(outlier_list_col) 
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(key for key, value in outlier_indices.items() if value > n) 
    return multiple_outliers

# outliers_to_drop = detect_outliers(train_copy, 2, ['Age', 'SibSp', 'Parch', 'Fare'])
# 
outliers_to_drop = detect_outliers(train_copy, 2, ['Age', 'FamilySize', 'Fare'])
print("We will drop these {} indices: ".format(len(outliers_to_drop)), outliers_to_drop)

In [ ]:
### train test split 